In [12]:
#Laad credentials and import relevant packages
import sys, os

# Make sure the current notebook's folder is searched first
notebook_dir = os.path.abspath("")
if notebook_dir not in sys.path:
    sys.path.insert(0, notebook_dir)

In [14]:
# from google.auth.transport.requests import Request
# from google.oauth2.service_account import Credentials

In [16]:
# Get credentials and project ID
# from utils import authenticate
# credentials, PROJECT_ID = authenticate()

In [20]:
# Create credentials object

# credentials = Credentials.from_service_account_file(
#     key_path,
#     scopes=['https://www.googleapis.com/auth/cloud-platform'])

# if credentials.expired:
#     credentials.refresh(Request())

In [22]:
# PROJECT_ID = 'chatbot-ai-463712'
# REGION = 'us-central1'

In [24]:
import pandas as pd
import numpy as np
# from vertexai.language_models import TextEmbeddingModel
# import vertexai

# # Initialize Vertex AI (replace with your project and region)
# vertexai.init(project=PROJECT_ID, location=REGION, credentials = credentials)

# model = TextEmbeddingModel.from_pretrained("text-embedding-005")

In [14]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer

df = pd.read_csv("BankFAQs.csv")
print("Shape: " + str(df.shape))

# Create embeddings of Questions and another embeddings of question and answers seperately
df.dropna(subset=['Question', 'Answer'], inplace=True)

#New coloumn for the combined embedding
df['Combined'] = df['Question'] + " " + df['Answer']

questions = df['Question'].tolist()
combined = df['Combined'].tolist()

# Load local model
local_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')  # 384-dim

# Generate embeddings
df['question_embedding'] = local_model.encode(questions, show_progress_bar=True).tolist()
df['combined_embedding'] = local_model.encode(combined, show_progress_bar=True).tolist()

# Save locally
df.to_pickle("bankfaqs_with_local_embeddings.pkl")

Shape: (1499, 3)


Batches:   0%|          | 0/47 [00:00<?, ?it/s]

Batches:   0%|          | 0/47 [00:00<?, ?it/s]

<class 'list'>
384


In [20]:
from sklearn.metrics.pairwise import cosine_similarity

df = pd.read_pickle("bankfaqs_with_local_embeddings.pkl")
model = SentenceTransformer('all-MiniLM-L6-v2')

def get_best_match(user_query: str, df: pd.DataFrame, model: SentenceTransformer, use_combined=False):
    # Embed user query
    query_embedding = model.encode([user_query])[0].reshape(1, -1)
    
    # Choose which embeddings to use
    if use_combined:
        corpus_embeddings = np.vstack(df['combined_embedding'].values)
    else:
        corpus_embeddings = np.vstack(df['question_embedding'].values)
    
    # Compute cosine similarity
    similarities = cosine_similarity(query_embedding, corpus_embeddings)[0]
    
    # Get the top result
    top_idx = np.argmax(similarities)
    matched_question = df.iloc[top_idx]['Question']
    matched_answer = df.iloc[top_idx]['Answer']
    score = similarities[top_idx]
    
    return matched_question, matched_answer, score


In [24]:
#Semantic Search
#Part1: Cosine Similarity of query and question
#Part2: if the query of consumer is vague, we will use the combined for added context

query = "How can I block my debit card if it's stolen?"
question, answer, score = get_best_match(query, df, model, use_combined=False)

print("Matched Question:", question)
print("Answer:", answer)
print("Similarity Score:", score)

Matched Question: What is the procedure to follow if my Debit Card PIN is blocked
Answer: Please note that if you enter an incorrect PIN three times in the ATM, your access gets blocked for security reasons. It gets activated after 24 hours. Kindly use your Debit / ATM Card at the ATM after 24 hours with the same PIN available with you. If your account still remains inaccessible, please apply for new PIN. You can apply for regeneration of your ATM / Debit PIN in following ways: Fill-up the new PIN application form and submit it at any of our branches. On receipt of the form and successful verification, we will process a new ATM PIN and send it to your mailing address, as registered in our records. Submit a request at PhoneBanking, after validation with your Telephone Identification Number (TIN). For every PIN Regeneration request, charges of Rs. 50 + Service Tax + Educational Cess is applicable. For more details, Contact Us View more
Similarity Score: 0.7117532768559258
